<h1 align="center"> Sensordatenfusion Tutorium 03 </h1>

<h3 align="center"> Erstellen der Prädiktion </h3>

Die Prädiktion hat den Zweck, dass wir eine Zustandsschätzung basierend auf $x_{k-1|k-1}$ erhalten.
Für $x_{k|k-1}$ benötigen wir lediglich noch unsere Evolutions-/Dynamikmatrix $F_{k|k-1}$, die im Zuge des zweiten Tutoriums erstellt wurde .
Für die dazugehörige Kovarianz $P_{k|k-1}$ brauchen wir zusätzlich zu $F$ noch das Prozessrauschen $Q_{k|k-1}$ (in den Folien $D_{k|k-1}$).
Ziel dieses Aufgabenteils ist es also, einen eigenen $Predictor$ zu erstellen, um $x_{k|k-1}$ und $P_{k|k-1}$ zu erhalten. 



In [ ]:
from functools import lru_cache
import scipy as sp
import numpy as np
from stonesoup.base import Property
from stonesoup.predictor import Predictor
from stonesoup.functions import gauss2sigma, unscented_transform
from stonesoup.types.prediction import GaussianStatePrediction
from stonesoup.types.state import State

Benötigt wird im Folgenden also die Transitionsmatrix $F_{k|k-1}$ = $\begin{bmatrix}
  1 & \Delta T_k  \\
  0 & 1  \\
\end{bmatrix}$ 
und das Rauschen $D_{k|k-1}$ = $\Sigma^2_k$ * $\begin{bmatrix}
  \frac{1}{4}\Delta T^4_k 1 & \frac{1}{2}\Delta T^3_k 1  \\
  \frac{1}{2}\Delta T^3_k 1 & \Delta T^2_k 1  \\
\end{bmatrix}$.

In [ ]:
class SdfKalmanPredictor(Predictor):
    @lru_cache()
    def predict(self, prior, control_input=None, timestamp=None, **kwargs):
        
        # Transition model parameters
        transition_matrix = self.transition_model.matrix
        transition_noise_covar = self.transition_model.covar
        
        # Perform prediction
        prediction_mean = transition_matrix @ prior.mean
        prediction_covar = transition_matrix @ prior.covar @ transition_matrix + transition_noise_covar
        
        return GaussianStatePrediction(prediction_mean,
                                       prediction_covar,
                                       timestamp)
    

Als Ausgabe erhalten sollten wir $x_{k|k-1}$ und $P_{k|k-1}$ erhalten, 
die im nächsten Tutorium dann auf $x_{k|k}$ und $P_{k|k}$ aktualisiert werden.

Als Ausgabe erhalten sollten wir $x_{k|k-1}$ und $P_{k|k-1}$ erhalten, 
die im nächsten Tutorium dann auf $x_{k|k}$ und $P_{k|k}$ aktualisiert werden.